# Setup Environment

In [ ]:
!pip install streamlit yfinance

In [ ]:
!mkdir -p src utils

In [ ]:
%%writefile utils/formatting.py
def format_number(num):
    """Formats a number with K/M/B suffixes."""
    if num is None or num == 'N/A':
        return 'N/A'
    try:
        num = float(num)
        if num >= 1_000_000_000:
            return f"{num/1_000_000_000:.2f}B"
        if num >= 1_000_000:
            return f"{num/1_000_000:.2f}M"
        if num >= 1_000:
            return f"{num/1_000:.2f}K"
        return f"{num:.2f}"
    except:
        return num


In [ ]:
%%writefile config.py
# Configuration for Warrior Trading Criteria and Application Settings

# Stock Selection Criteria
MIN_PRICE = 1.00
MAX_PRICE = 20.00
MIN_DAY_CHANGE_PCT = 10.0
MIN_RELATIVE_VOLUME = 5.0

# Application Settings
NUMBER_OF_TICKERS_TO_SCAN = 100
TICKER_SOURCE_URL = "https://raw.githubusercontent.com/rreichel3/US-Stock-Symbols/main/all/all_tickers.txt"


In [ ]:
%%writefile src/data.py
import yfinance as yf
import pandas as pd
import requests
import random
from datetime import datetime
from config import NUMBER_OF_TICKERS_TO_SCAN, TICKER_SOURCE_URL
from utils.formatting import format_number

def get_random_tickers(n=NUMBER_OF_TICKERS_TO_SCAN):
    """Fetches a list of US tickers and returns a random sample."""
    try:
        response = requests.get(TICKER_SOURCE_URL)
        response.raise_for_status()
        tickers = response.text.splitlines()
        tickers = [t.strip() for t in tickers if t.strip()]
        sample_size = min(n, len(tickers))
        return random.sample(tickers, sample_size)
    except Exception as e:
        print(f"Error fetching ticker list: {e}")
        return ["AAPL", "MSFT", "AMZN", "NVDA", "TSLA", "GME", "AMC", "BB", "PLTR", "SOFI"]

def fetch_market_data(tickers):
    """
    Fetches current market data for the given tickers using yfinance.
    """
    data = []
    try:
        # Fetch 1mo history
        history = yf.download(tickers, period="1mo", group_by='ticker', progress=False, threads=True)
        
        current_time = datetime.now().strftime("%H:%M:%S")

        for ticker in tickers:
            try:
                if len(tickers) == 1:
                    df_ticker = history
                else:
                    if ticker not in history.columns.levels[0]:
                        continue
                    df_ticker = history[ticker]
                
                if df_ticker.empty:
                    continue
                
                latest = df_ticker.iloc[-1]
                prev = df_ticker.iloc[-2] if len(df_ticker) > 1 else latest
                
                price = latest['Close']
                prev_close = prev['Close']
                volume = latest['Volume']
                
                change_pct = ((price - prev_close) / prev_close) * 100
                avg_volume = df_ticker['Volume'].mean()
                rel_vol = volume / avg_volume if avg_volume > 0 else 0
                
                data.append({
                    'Symbol': ticker,
                    'Time': current_time,
                    'Price': float(price),
                    'Change %': float(change_pct),
                    'Volume': int(volume),
                    'Avg Volume': int(avg_volume),
                    'Rel Volume': float(rel_vol),
                    'Float': 'N/A' # Placeholder, will enrich later
                })
                
            except Exception as e:
                continue
                
    except Exception as e:
        print(f"Batch download error: {e}")
        
    return pd.DataFrame(data)

def enrich_with_float(df):
    """
    Fetches Float data for the filtered DataFrame. 
    """
    floats = []
    for symbol in df['Symbol']:
        try:
            info = yf.Ticker(symbol).info
            f = info.get('floatShares', None)
            floats.append(format_number(f) if f else 'N/A')
        except:
            floats.append('N/A')
    df['Float'] = floats
    return df


In [ ]:
%%writefile src/news.py
import yfinance as yf

def get_catalysts(symbol):
    """
    Fetches latest news for a specific stock symbol to identify catalysts.
    """
    try:
        ticker = yf.Ticker(symbol)
        news_items = ticker.news
        
        formatted_news = []
        for item in news_items:
            formatted_news.append({
                'Title': item.get('title'),
                'Publisher': item.get('publisher'),
                'Link': item.get('link'),
                'Time': item.get('providerPublishTime')
            })
        return formatted_news
    except Exception as e:
        return []

def get_latest_headline(symbol):
    """
    Fetches the latest news headline as a catalyst summary (approx < 50 words).
    """
    try:
        news = get_catalysts(symbol)
        if news:
            latest = news[0]
            # Create a summary string
            summary = f"{latest['Title']} ({latest['Publisher']})"
            # Truncate if too long (rough word count)
            words = summary.split()
            if len(words) > 50:
                summary = " ".join(words[:50]) + "..."
            return summary
    except:
        pass
    return "No recent news"


In [ ]:
%%writefile src/screener.py
from config import MIN_PRICE, MAX_PRICE, MIN_DAY_CHANGE_PCT, MIN_RELATIVE_VOLUME

def filter_stocks(df):
    """
    Applies Warrior Trading criteria to the dataframe.
    """
    if df.empty:
        return df

    # 1. Price Range ($1.00 - $20.00)
    df_filtered = df[(df['Price'] >= MIN_PRICE) & (df['Price'] <= MAX_PRICE)].copy()
    
    # 2. Percentage Change (>= 10%)
    df_filtered = df_filtered[df_filtered['Change %'] >= MIN_DAY_CHANGE_PCT]
    
    # 3. Relative Volume (>= 5x)
    df_filtered = df_filtered[df_filtered['Rel Volume'] >= MIN_RELATIVE_VOLUME]
    
    return df_filtered


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from src.data import get_random_tickers, fetch_market_data, enrich_with_float
from src.screener import filter_stocks
from src.news import get_latest_headline
from config import NUMBER_OF_TICKERS_TO_SCAN
from utils.formatting import format_number

st.set_page_config(page_title="Warrior Stock Screener", layout="wide")

st.markdown("""
<style>
    .stDataFrame {font-size: 14px;}
</style>
""", unsafe_allow_html=True)

st.title("🛡️ Warrior Trading Stock Screener")

# Sidebar settings
st.sidebar.header("Scanner Settings")
n_tickers = st.sidebar.slider("Sample Size (random tickers)", 10, 500, NUMBER_OF_TICKERS_TO_SCAN)

if st.sidebar.button("Run Scanner") or "scan_results" not in st.session_state:
    with st.spinner(f"Scanning {n_tickers} random tickers..."):
        # 1. Get Tickers & Data
        tickers = get_random_tickers(n_tickers)
        df_market = fetch_market_data(tickers)
        
        if not df_market.empty:
            st.session_state['scan_results'] = df_market
            
            # 2. Filter
            filtered_df = filter_stocks(df_market)
            
            # 3. Enrich Candidates (Float + Catalyst)
            if not filtered_df.empty:
                st.info(f"Found {len(filtered_df)} matches. Enriching data...")
                filtered_df = enrich_with_float(filtered_df)
                
                # Fetch Catalysts
                catalysts = []
                for sym in filtered_df['Symbol']:
                    catalysts.append(get_latest_headline(sym))
                filtered_df['Catalyst'] = catalysts
                
            st.session_state['filtered_results'] = filtered_df
        else:
            st.error("No market data fetched.")

# Display Results
if 'filtered_results' in st.session_state:
    df_res = st.session_state['filtered_results']
    
    col1, col2 = st.columns([3, 1])
    
    with col1:
        st.subheader("Top Gainers (Warrior Criteria Match)")
        if not df_res.empty:
            # Reorder columns to match request roughly
            # Change %, Symbol, Price, Volume, Float, Rel Volume, Time, Catalyst
            display_cols = ['Change %', 'Symbol', 'Price', 'Volume', 'Float', 'Rel Volume', 'Time', 'Catalyst']
            df_display = df_res[display_cols].copy()
            
            # Format large numbers for display (Volume is int, change to string formatted)
            df_display['Volume'] = df_display['Volume'].apply(format_number)
            
            # Styling
            st.dataframe(
                df_display.style
                .format({
                    'Price': '${:.2f}',
                    'Change %': '{:.2f}',
                    'Rel Volume': '{:.2f}'
                })
                .background_gradient(subset=['Change %'], cmap='RdYlGn', vmin=0)
                .background_gradient(subset=['Rel Volume'], cmap='Blues')
                .set_properties(**{'text-align': 'center'})
            , use_container_width=True)
        else:
            st.warning("No matches found in this batch.")
            
    with col2:
        st.subheader("All Scanned (Preview)")
        df_all = st.session_state.get('scan_results', pd.DataFrame())
        if not df_all.empty:
            st.dataframe(
                df_all[['Symbol', 'Change %', 'Price', 'Volume']]
                .sort_values('Change %', ascending=False)
                .head(20)
                .style.format({'Change %': '{:.2f}', 'Price': '${:.2f}'})
                .background_gradient(subset=['Change %'], cmap='RdYlGn'),
                use_container_width=True
            )


# Run Streamlit
Run the cell below. It will install localtunnel and provide a link to view the app.

In [ ]:

import subprocess
import time

# Run streamlit in the background
print("Starting Streamlit...")
process = subprocess.Popen(["streamlit", "run", "app.py"])

# Use localtunnel to expose the port (alternative to ngrok)
print("Installing localtunnel...")
!npm install -g localtunnel

print("Starting Tunnel... Click the url below (and maybe enter the public IP if asked)")
# We use curl to get the public IP to help the user if localtunnel asks for password
!curl ipv4.icanhazip.com
!npx localtunnel --port 8501
